## Tutorial: Gene annotation comparison

In this example, we compare two human gene annotation sets, namely gencode (v39) and chess (v3.0.1).
We load all protein_coding genes from both annotation sets, update the gene-names to their latest given symbol (using gennames.org data) and compare both lists. We then exemplary show how to further analyze differences between those gene annotation sets.

*Required resources:*
- Human genome FASTA (GRCh38), accessible at https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_000001405.26/
- Full gencode annotation gff3 file (sorted), available at https://www.gencodegenes.org/human/
- Full chess 3.0.1 annotation gff3 file (sorted), available at https://github.com/chess-genome/chess/releases/download/v.3.0.1/chess3.0.1.gff.gz

In [ ]:
# set path and load rnalib
import os, pathlib, platform
rnalib_SRC=pathlib.Path('/Users/niko/projects/rnalib/') 
os.chdir(rnalib_SRC)
# install libraries. Recommended to run in a venv here!
#!{sys.executable} -m pip install -r requirements.txt 
display(f"Running rnalib on python {platform.python_version()}. Using rnalib code from {rnalib_SRC}")
# load rnalib
import rnalib as pg
from rnalib import SEP, display_textarea
from tqdm.auto import tqdm
import tempfile
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import pandas as pd
import numpy as np
import traceback
import math

First, we download gencode 39 and chess 3.0.1 annotations as well as a table with current gene symbols and their aliases from genenames.org.
NOTE that this needs bedtools, samtools and htslib (bgzip, tabix) installed.
Total size of the downloaded data (for all tutorials) is ~150M. Files are only downloaded if not existing already in the `notebooks/large_test_resources/` directory.

In [ ]:
outdir=rnalib_SRC / 'notebooks/large_test_resources' # update to your preferred location
large_test_resources = {
    "outdir": f"{outdir}", # update to your preferred location
    "resources": {
        # -------------- Full gencode39 annotation -------------------------------
        "full_gencode_gff": {
            "uri": "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_39/gencode.v39.annotation.gff3.gz",
            "filename": "gencode_39.gff3.gz",
            "recreate": False
        },
        # -------------- Full chess annotation -------------------------------
        "full_chess_gtf": {
            "uri": "https://github.com/chess-genome/chess/raw/master/chess3.0.1.gtf.gz",
            "filename": "chess3.0.1.gtf.gz",
            "recreate": False
        },
        # -------------- Gene name aliases -------------------------------
        "gene_aliases": {
            "uri": "https://ftp.ebi.ac.uk/pub/databases/genenames/hgnc/tsv/hgnc_complete_set.txt",
            "filename": "hgnc_complete_set.txt",
            "recreate": False
        },
        # -------------- GRCh38 chr20 -------------------------------
        "grch38_chr20": {
            "uri": "https://hgdownload.cse.ucsc.edu/goldenpath/hg38/chromosomes/chr20.fa.gz",
            "filename": "grch38_chr20.fa.gz",
            "recreate": False
        }
    }
}
display(f'Downloading test data files to {outdir}')
for resname in large_test_resources['resources']:
    try:
        pg.testdata.download_bgzip_slice(large_test_resources, resname, view_tempdir=False)
    except Exception:
        display(traceback.format_exc())
        display(f"Error creating resource {resname}. Some tests may not work...")
display("All done.")

Now we load gencode and chess annotation data and build the respective transcriptomes (takes about 4min).
We configure a transcript filter that includes only entries with gene_type 'protein_coding' and restrict this analysis to all canonical GRCh38 chromsomes (i.e., chr1-22,X,Y,M).
To update to the latest gene symbols, we use genenames.org data and configure via the 'gene_name_alias_file' property. rnalib automatically updates gene names accordingly. 

In [ ]:
alias_txt=pg.get_resource("gene_aliases", conf=large_test_resources) # gene name aliases
gencode_gff=pg.get_resource("full_gencode_gff", conf=large_test_resources) # Gencode data
chess_gff=pg.get_resource("full_chess_gtf", conf=large_test_resources) # Chess data

txfilter = pg.TranscriptFilter(). \
    include_gene_types({'protein_coding'}). \
    include_chromosomes(pg.CANONICAL_CHROMOSOMES['GRCh38'])

t_gc=pg.Transcriptome({
    'annotation_gff': gencode_gff,
    'annotation_flavour': 'gencode',
    'copied_fields': ['gene_type', 'transcript_type'],
    'drop_empty_genes': False,
    'gene_name_alias_file': alias_txt
}, txfilter)
# NOTE: in chess, there ais no 'transcript_type' annotation
t_ch=pg.Transcriptome({
    'annotation_gff': chess_gff,
    'annotation_flavour': 'chess',
    'copied_fields': ['gene_type'],
    'drop_empty_genes': False,
    'gene_name_alias_file': alias_txt
}, txfilter)
# compare
display(SEP)
display(t_gc)
display(t_ch)

Please note that, the numbers reported in the [Chess3 paper](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-023-03088-4/) differ, particularly in the number of protein_coding gencode transcripts:

| Database    | Number of protein-coding gene loci | Number of protein-coding transcripts | Number of distinct protein sequences | Number of gene loci (all types) |
|-------------|------------------------------------|--------------------------------------|--------------------------------------|---------------------------------|
| CHESS v3    | 19,839                             | 99,202                               | 73,767                               | 41,356                          |
| RefSeq v110 | 19,884                             | 129,740                              | 88,662                               | 43,380                          |
| GENCODE v41 | 19,419                             | 110,309                              | 92,968                               | 46,181                          |

*Table 1 from the [CHESS3 paper](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-023-03088-4/tables/1): Total number of genes and protein-coding isoforms in current versions of CHESS, RefSeq, and GENCODE. Genes are counted on the primary chromosomes and unplaced scaffolds from the human reference genome GRCh38, excluding the alternative scaffolds. Pseudogenes, VDJ segments, and C regions are not included in the totals shown in the final column.*

This can partially be explained by the different chromosome sets used (we include only primary chromosomes).

Now we can have a look at the shared and unique (normalized) gene names and
display the unique gene names for genocde and chess.

In [ ]:
# cmp_sets is a rnalib utility function for set comparison
shared, uniq_gc, uniq_ch = pg.cmp_sets({x.gene_name for x in t_gc.genes}, 
                                       {x.gene_name for x in t_ch.genes}) 
display(f"Unique gene names in gencode: {len(shared)+len(uniq_gc)}, in chess: {len(shared)+len(uniq_ch)}")
display(f"Shared: {len(shared)}, only in gencode: {len(uniq_gc)}, only in chess: {len(uniq_ch)}")
display_textarea(f"Unique in gencode: {uniq_gc}")
display_textarea(f"Unique in chess: {uniq_ch}")

As expected, most genes are annotated in both sets as expected but there are some unique entries too.
Most unique genecode genes have no given name but an ensembl gene id (starting with ENSG), 
while most unique chess genes start with “LOC” which indicates unknown function.
Let's have a look at 'GCNT7' which is only annotated in gencode 
(https://www.genecards.org/cgi-bin/carddisp.pl?gene=GCNT7)
 We query the chess annotation set for the coordinates of the GCNT7 gene in the gencode annotation:

In [ ]:
display(f"In Gencode, GCNT7 is annotated at {t_gc.gene['GCNT7'].location}")
display("In Gencode at this region:", [(x.location, x.gene_name, x.strand) for x in t_gc.query(t_gc.gene['GCNT7'], feature_types=['gene'])])
display("In Chess at this region:", [(x.location, x.gene_name, x.strand) for x in t_ch.query(t_gc.gene['GCNT7'], feature_types=['gene'])])

Indeed, it shows only RTF2 and 'FAM209A', both on the opposite strand.
Vice versa, we look at 'MMP24-AS1-EDEM2' which is found only in the chess annotation

In [ ]:
display(f"In Chess, MMP24-AS1-EDEM2 is annotated at {t_ch.gene['MMP24-AS1-EDEM2'].location}")
display("In Gencode at this region:", [(x.location, x.gene_name, x.strand) for x in t_gc.query(t_ch.gene['MMP24-AS1-EDEM2'], feature_types=['gene'])])
display("In Chess at this region:", [(x.location, x.gene_name, x.strand) for x in t_ch.query(t_ch.gene['MMP24-AS1-EDEM2'], feature_types=['gene'])])

Note that not all gene names in a transcriptome are unique. For example, genes in pseudo-autosomal 
regions (PAR), such as WASH6P, are annotated at different genomic locations (e.g., X and Y chromosomes)
with the same gene_name but different feature IDs. rnalib properly handles such
genes and maintains a dict that maps duplicate gene_names to the respective gene objects:

In [ ]:
display_textarea(f"Duplicate gene names in gencode: {t_gc.duplicate_gene_names}") 
display_textarea(f"Duplicate gene names in chess: {t_ch.duplicate_gene_names}")

For a more detailed analysis, we iterate all gencode genes and overlap with chess annotations 
using rnalib's annotation iterator. 
We count some stats and select the chess annotation with the highest overlap for a gene-to-gene comparison.
Finally, we calculate a histogram of absolute difference between gencode and chess annotations for start and end 
coordinates and plot the first 50 values in a barplot

In [ ]:
stats=Counter()
examples=dict()
hist_start, hist_end=Counter(), Counter()
diff_annos=list()
with pg.AnnotationIterator(
    pg.TranscriptomeIterator(t_gc, feature_types='gene'),
    pg.TranscriptomeIterator(t_ch, feature_types='gene')) as it:
    for gene_gc, dat in it:
        stats['gencode_genes']+=1
        if len(dat.it0)==0:
            stats['gencode_genes_no_match']+=1
            continue
        genes_ch = {g.location for g in dat.it0}
        # check whether gene names match
        if gene_gc.gene_name not in [g.gene_name for g in genes_ch]:
            stats['differing_gene_name'] += 1
            if gene_gc.gene_name not in shared:
                stats['differing_gene_name_uniq'] += 1
            examples['differing_gene_name_example'] = f"gc: {gene_gc.gene_name}, ch: {[g.gene_name for g in genes_ch]}"
            continue
        # check whether coordinates match
        if gene_gc.get_location() not in [g.get_location() for g in genes_ch]:
            stats['differing_location'] += 1
            examples['differing_location_example'] = f"gc: {gene_gc.get_location()}, ch: {genes_ch}"
            # select overlapping chess annotation with minimum abs difference
            overlaps={g:abs(gene_gc.left_pos().distance(g.left_pos()))+abs(gene_gc.right_pos().distance(g.right_pos())) for g in genes_ch}
            closest_anno=min(overlaps, key=overlaps.get)
            abs_diff_start = abs(gene_gc.left_pos().distance(closest_anno.left_pos()))
            abs_diff_end = abs(gene_gc.right_pos().distance(closest_anno.right_pos()))
            hist_start[abs_diff_start]+=1
            hist_end[abs_diff_end]+=1
            diff_annos.append((abs_diff_start+abs_diff_end,gene_gc, closest_anno))
display(stats)
display(examples)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
x,y=zip(*sorted(hist_start.items())[:50])
ax1.bar(x,y)
ax1.set_xlabel('start difference')
x,y=zip(*sorted(hist_end.items())[:50])
ax2.bar(x,y)
ax2.set_xlabel('end difference')
fig.suptitle(t='Difference to gencode annotations', fontsize=16)
fig.tight_layout()

So most annotations indeed differ between gencode and chess and the differing ones share 
either the same start or the same end coordinates. 
There are, however, some annotations with very large differences (multiple Mbases) that are not shown in the barplots. Let's have a closer look at the 10 most extreme examples:

In [ ]:
for diff, g_gc, g_ch in sorted(diff_annos, reverse=True)[:10]:
    display(f"{diff}, {len(g_gc)}, {g_gc.gene_name}, {g_gc.get_location()}, {g_ch.gene_name}, {g_ch.get_location()}")

Finally, lets have a look at the number of annotated transcripts per annotation set.
For this analysis, we consider only the shared genes

In [ ]:
# calculate vectors of pairwise transcript counts for all shared gene_names
x, y = zip(*[(len(t_gc.gene[gn].transcript), len(t_ch.gene[gn].transcript) ) for gn in shared ])
# scatter plot
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
plt.hexbin(x,y, bins='log')
ax.axline((1, 1), slope=1)
plt.xlabel('Gencode')
plt.ylabel('Chess')
# correlation coefficient
display(f'R_pearson={np.corrcoef(x,y)[0,1]}')

We can observe large differences between the compared annotation set. Let's
look at the most extreme example.

In [ ]:
max_diff_idx = np.argmax(np.array([abs(a-b) for a,b in zip(x,y)]))
max_diff_gene = list(shared)[max_diff_idx]
display(f"The gene with the largest difference in annotated transcripts is {t_gc.gene[max_diff_gene].gene_name}")

tr_gc, tr_ch = t_gc.gene[max_diff_gene].transcript, t_ch.gene[max_diff_gene].transcript
display(f"In gencode, there are {len(tr_gc)} tx, in chess, there are {len(tr_ch)}")
display_textarea(f"gencode tx: {tr_gc}")
display_textarea(f"chess tx: {tr_ch}")

In a final analysis, we want to count the number of introns with canonical splice sites, i.e.,
having GT and AG at either end of that intron. For this, we must load gene sequences and in this
example we do this only for chr20. 

First, we instantiate 2 new transcriptomes that use the same configuration as before but now add the grch38_chr20 
reference sequence and load those sequences at startup. Note that rnalib detects that the referenced genome contains
only chr20 and will merge the respective refdicts resulting in only genes/transcripts located on chr20 being loaded.

In [ ]:
grch38_chr20 = pg.get_resource("grch38_chr20", conf=large_test_resources) # GRCh38 chr20 reference sequence
# Instantiate a gencode/chr20 transcriptome and reuse configuration from before 
t_gc_chr20=pg.Transcriptome(
    dict( t_gc.config, **{'genome_fa': grch38_chr20, 'load_sequences': True})
)
# Instantiate a chess/chr20 transcriptome and reuse configuration from before 
t_ch_chr20=pg.Transcriptome(
    dict( t_ch.config, **{'genome_fa': grch38_chr20, 'load_sequences': True})
)
# show merged refdicts and numbers of loaded genes/transcripts
display(SEP)
display(t_gc_chr20, t_gc_chr20.merged_refdict)
display(t_ch_chr20, t_ch_chr20.merged_refdict)

In [ ]:
# now, lets count introns with and without consensus dinucleotides (GT and AG) 
# at either end of that intron
stats = defaultdict(Counter)
for name, tr in [('gencode',t_gc_chr20),('chess', t_ch_chr20)]:
    for intron, _ in pg.TranscriptomeIterator(tr, feature_types='intron'):
        if len(intron)<4:
            continue
        a,b = intron.sequence[:2].upper(), intron.sequence[-2:].upper()
        if intron.strand=='-': # reverse complement for minus strand genes
            a,b = pg.reverse_complement(b), pg.reverse_complement(a)
        stats[name][f'{a}/{b}'] += 1
# convert to dataframe
df = pd.DataFrame.from_records([(n,t,c) for n in stats for t,c in stats[n].items()], 
                               columns=['annotation', 'SJ', 'count']).sort_values(['count'], ascending=False)
display(df.head())

# Show % canonical SJ
# compare: PMID: 11058137: 98.71% canonical, 0.56% non-canonical GC-AG
display(f"In gencode, we found {stats['gencode']['GT/AG'] / stats['gencode'].total() * 100}% canonical splice sites")
display(f"In chess,   we found {stats['chess']['GT/AG'] / stats['chess'].total() * 100}% canonical splice sites")

# plot results
sns.barplot(x='SJ', y='count', data=df, hue='annotation')
_= plt.xticks(rotation=45)
_= plt.yscale("log")


So, gencode shows more non-canonical splice junctions that are possibly wrong.